<a href="https://colab.research.google.com/github/AnastasiiaMois/NLP/blob/master/%D0%AF%D0%BD%D0%B5%D0%BD%D0%B0%D0%B2%D0%B8%D0%B6%D1%83%D1%81%D0%B2%D0%BE%D1%8E%D0%B6%D0%B8%D0%B7%D0%BD%D1%8C%D0%B8%D0%B2%D1%81%D0%B5%D1%8D%D1%82%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install fasttext

     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2384115 sha256=856d22ee651b6886ff3f417705454182c20083b008f44d5436300316099d19ef
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import fasttext
import bz2
import csv
from sklearn.metrics import roc_auc_score
import os

In [22]:
data = bz2.BZ2File("train.ft.txt.bz2")
data = data.readlines()
data = [x.decode('utf-8') for x in data]
print(len(data))

3600000


In [7]:
data = pd.DataFrame(data)
data.to_csv("train.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

# Modelling
# This routine takes about 5 to 10 minutes 
model = fasttext.train_supervised('train.txt',label_prefix='__label__', thread=4, epoch = 10)
print(model.labels, 'are the labels or targets the model is predicting')

['__label__1', '__label__2'] are the labels or targets the model is predicting


In [9]:
test = bz2.BZ2File("test.ft.txt.bz2")
test = test.readlines()
test = [x.decode('utf-8') for x in test]
print(len(test), 'number of records in the test set') 

# To run the predict function, we need to remove the __label__1 and __label__2 from the testset.  
new = [w.replace('__label__2 ', '') for w in test]
new = [w.replace('__label__1 ', '') for w in new]
new = [w.replace('\n', '') for w in new]

# Use the predict function 
pred = model.predict(new)

# check the first record outputs
print(pred[0][0], 'is the predicted label')
print(pred[0][1], 'is the probability score')

400000 number of records in the test set
['__label__2'] is the predicted label
['__label__2'] is the probability score


In [10]:
labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test]
pred_labels = [0 if x == ['__label__1'] else 1 for x in pred[0]]

# run the accuracy measure. 
print(roc_auc_score(labels, pred_labels))

0.91735


In [11]:
data[:4]

,0
0,__label__2 Stuning even for the non-gamer: Thi...
1,__label__2 The best soundtrack ever to anythin...
2,__label__2 Amazing!: This soundtrack is my fav...
3,__label__2 Excellent Soundtrack: I truly like ...


In [0]:
train_labels  = [0 if x.split(' ')[0] == '__label__1' else 1 for x in data]

In [0]:
new_train = [w.replace('__label__2 ', '') for w in data]
new_train = [w.replace('__label__1 ', '') for w in new]
new_train = [w.replace('\n', '') for w in new]

In [0]:
data_train = pd.DataFrame(new_train)

In [30]:
data_train[:4]

,0
0,Great CD: My lovely Pat has one of the GREAT v...
1,One of the best game music soundtracks - for a...
2,Batteries died within a year ...: I bought thi...
3,"works fine, but Maha Energy is better: Check o..."


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix

In [0]:
pipeline = Pipeline([
    ('bow',CountVectorizer()), # converts strings to integer counts
    ('tfidf',TfidfTransformer()), # converts integer counts to weighted TF-IDF scores
    ('classifier',MultinomialNB()) # train on TF-IDF vectors with Naive Bayes classifier
])

In [0]:
data_train_clean = data_train.copy()

In [0]:
data_train_clean[0] = data_train_clean[0].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip()

In [0]:
data_train_clean[0] = data_train_clean[0].str.lower()

In [0]:
data_train_clean[0] = data_train_clean[0].str.split()

In [39]:
data_train_clean.head()

,0
0,"[great, cd, my, lovely, pat, has, one, of, the..."
1,"[one, of, the, best, game, music, soundtracks,..."
2,"[batteries, died, within, a, year, i, bought, ..."
3,"[works, fine, but, maha, energy, is, better, c..."
4,"[great, for, the, non, audiophile, reviewed, q..."


In [42]:
pipeline.fit(data_train_clean,labels_tr)

AttributeError: ignored

In [0]:
labels_tr = pd.DataFrame(train_labels)

In [43]:
labels_tr.head()

,0
0,1
1,1
2,1
3,1
4,1


In [0]:
from sklearn.feature_extraction.text import CountVectorizer